In [ ]:
import networkx as nx 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import tensorflow as tf 
import tensorflow_gnn as tfgnn
import networkx as nx

import ipynb.fs
from ipynb.fs.full.graph_features import build_fmatrix, build_subgraph, agg_feat
from ipynb.fs.full.make_graphs import node_attribute, create_graph

import warnings
warnings.simplefilter("ignore")

node_attrib = node_attribute()
feat_matrix = build_fmatrix()
feat_list = agg_feat() 
fl = len(feat_list)

In [ ]:
tf.random.set_seed(42)

def graph_tfgnn(labelz, H):
    tf.keras.backend.clear_session()
    
    num_node = len(list(H.nodes))
    # plt.title('Number Nodes: {}'.format(num_node))
    # nx.draw_networkx(H, with_labels=True, node_size=50)
    # nx.draw_networkx(H, pos=nx.spring_layout(H))
    # plt.show()
    
    #BuildingEdgeDF
    def edgeview():
        init_edge_df = pd.DataFrame(H.edges, columns=['source', 'target'])
        
        rev = {'source':list(init_edge_df['target']),
                  'target':list(init_edge_df['source'])}
        reverse_df = pd.DataFrame(rev, columns=['source', 'target'])
        
        edge_df = pd.concat([init_edge_df,reverse_df], ignore_index=True, axis=0)
        
        edge_attrib = []
        for i in range(len(edge_df)):
            a, b = edge_df.iloc[i]
            fmat = round(feat_matrix[a][b], 4)
            edge_attrib.append(fmat)
        
        edge_df['edge_att'] = edge_attrib
        edge_df['weight'] = [1 for i in range(len(edge_df))]
        return edge_df
        
    #BuildingNodeDF
    def nodeview():
        lb = labelz
        node_list = list(H.nodes)
        node_df = pd.DataFrame(node_list, columns=['nodelist'])
        att_list = [node_attrib[n] for n in node_list]
        
        node_df['node_att'] = att_list
        node_df['node_label'] = [lb for i in range(len(node_list))]
        
        return node_df
    
    def create_adj_id(node_df,edge_df):
        node_df = node_df.reset_index()
        edge_df = pd.merge(edge_df,node_df[['nodelist','index']].rename(columns={"index":"source_id"}),
                           how='left',left_on='source',right_on='nodelist').drop(columns=['nodelist'])
        edge_df = pd.merge(edge_df,node_df[['nodelist','index']].rename(columns={"index":"target_id"}),
                           how='left',left_on='target',right_on='nodelist').drop(columns=['nodelist'])
        
        edge_df.dropna(inplace=True)
        return node_df, edge_df
    
    full_node_df = nodeview()
    full_edge_df = edgeview()
    node_df_adj, edge_df_adj = create_adj_id(full_node_df, full_edge_df)
    
    def create_graph_tensor(node_df,edge_df):
        graph_tensor = tfgnn.GraphTensor.from_pieces(
            node_sets = {
                "nodes": tfgnn.NodeSet.from_fields(
                    sizes = [len(node_df)],
                    features ={
                        'node_att': np.array(node_df['node_att'], dtype='float32').reshape(len(node_df),1),
                        'node_label': np.array(node_df['node_label'], dtype='int32').reshape(len(node_df),1),
                    }),
            },
            edge_sets ={
                "link": tfgnn.EdgeSet.from_fields(
                    sizes = [len(edge_df)],
                    features = {
                        'edge_att': np.array(edge_df['edge_att'], dtype='float32').reshape(len(edge_df),1),
                        'weight': np.array(edge_df['weight'], dtype='int32').reshape(len(edge_df),1)
                    },
                    adjacency = tfgnn.Adjacency.from_indices(
                        source = ("nodes", np.array(edge_df['source_id'], dtype='int32')),
                        target = ("nodes", np.array(edge_df['target_id'], dtype='int32')),
                    )),
            })
        return graph_tensor 
    
     
    full_tensor = create_graph_tensor(node_df_adj, edge_df_adj)  
    
    def node_batch_merge(graph):
        graph = graph.merge_batch_to_components()
        node_features = graph.node_sets['nodes'].get_features_dict()
        edge_features = graph.edge_sets['link'].get_features_dict()
        
        label = node_features.pop('node_label')
        _ = edge_features.pop('weight')
        
        new_graph = graph.replace_features(
            node_sets={'nodes':node_features},
            edge_sets={'link':edge_features})
        return new_graph, label
    
    
    def edge_batch_merge(graph):
        graph = graph.merge_batch_to_components()
        node_features = graph.node_sets['nodes'].get_features_dict()
        edge_features = graph.edge_sets['link'].get_features_dict()
        
        _ = node_features.pop('node_label')
        label = edge_features.pop('weight')
        
        new_graph = graph.replace_features(
            node_sets={'nodes':node_features},
            edge_sets={'link':edge_features})
        return new_graph, label
    
    
    def create_dataset(graph,function):
        dataset = tf.data.Dataset.from_tensors(graph)
        dataset = dataset.batch(16)
        return dataset.map(function)
    
    #Node Datasets
    full_node_dataset = create_dataset(full_tensor, node_batch_merge)
    
    #Edge Datasets
    full_edge_dataset = create_dataset(full_tensor, edge_batch_merge)
    
    graph_spec = full_node_dataset.element_spec[0]
    input_graph = tf.keras.layers.Input(type_spec=graph_spec)
    
    def set_initial_node_state(node_set, node_set_name):
        features = [
            tf.keras.layers.Dense(16,activation="relu")(node_set['node_att'])    #32
        ]
        return tf.keras.layers.Concatenate()(features)
    
    def set_initial_edge_state(edge_set, edge_set_name):
        features = [
            tf.keras.layers.Dense(16,activation="relu")(edge_set['edge_att'])    #32
        ]
        return tf.keras.layers.Concatenate()(features)
    
    graph = tfgnn.keras.layers.MapFeatures(
        node_sets_fn=set_initial_node_state,
        edge_sets_fn=set_initial_edge_state
    )(input_graph)
    
    def dense_layer(units=16,l2_reg=0.1,dropout=0.2,activation='relu'):         #units=64
        regularizer = tf.keras.regularizers.l2(l2_reg)
        return tf.keras.Sequential([
            tf.keras.layers.Dense(units,
                                  kernel_regularizer=regularizer,
                                  bias_regularizer=regularizer),
            tf.keras.layers.Dropout(dropout)])
    
    #BuildingNodeModel
    graph_updates = 3 # tunable parameter
    for i in range(graph_updates):
        graph = tfgnn.keras.layers.GraphUpdate(
            node_sets = {
                'nodes': tfgnn.keras.layers.NodeSetUpdate({
                    'link': tfgnn.keras.layers.SimpleConv(
                        message_fn = dense_layer(16),                            #32
                        reduce_type="sum",
                        sender_edge_feature = tfgnn.HIDDEN_STATE,
                        receiver_tag=tfgnn.TARGET)},
                    tfgnn.keras.layers.NextStateFromConcat(
                        dense_layer(16)))})(graph)                              #64
        
        logits = tf.keras.layers.Dense(16,activation='sigmoid')(graph.node_sets["nodes"][tfgnn.HIDDEN_STATE])  #216 or fl
    
    node_model = tf.keras.Model(input_graph, logits)
    
    node_model.compile(
        tf.keras.optimizers.Adam(),
        loss = tf.keras.losses.mse,
        metrics = ['mse']
    )
    
    node_model.summary()
    
    es = tf.keras.callbacks.EarlyStopping(
            monitor='loss',mode='min',verbose=1,
            patience=3,restore_best_weights=True)
    
    node_model.fit(full_node_dataset.repeat(),
                   validation_data=full_node_dataset,
                   steps_per_epoch=10,
                   epochs=50,
                   callbacks=[es])
    
    
    #BuildingEdgemodel
    graph_spec_edge = full_edge_dataset.element_spec[0]
    input_graph_edge = tf.keras.layers.Input(type_spec=graph_spec_edge)
    graph_edge = tfgnn.keras.layers.MapFeatures(
        node_sets_fn=set_initial_node_state,
        edge_sets_fn=set_initial_edge_state
    )(input_graph_edge)
    
    graph_updates = 3
    for i in range(graph_updates):
        graph_edge = tfgnn.keras.layers.GraphUpdate(
            edge_sets = {'link': tfgnn.keras.layers.EdgeSetUpdate(
                next_state = tfgnn.keras.layers.NextStateFromConcat(
                    dense_layer(16,activation='relu')))},                       #32
            node_sets = {
                'nodes': tfgnn.keras.layers.NodeSetUpdate({
                    'link': tfgnn.keras.layers.Pool(
                        tag=tfgnn.TARGET,
                        reduce_type="sum",
                        feature_name = tfgnn.HIDDEN_STATE)},
                    tfgnn.keras.layers.NextStateFromConcat(
                        dense_layer(16)))})(graph_edge)                         #64
    
        logits = tf.keras.layers.Dense(16,activation='sigmoid')(graph_edge.edge_sets['link'][tfgnn.HIDDEN_STATE])  #216 or fl
    
    edge_model = tf.keras.Model(input_graph_edge, logits)
    
    edge_model.compile(
        tf.keras.optimizers.Adam(),
        loss = tf.keras.losses.mse,
        metrics = ['mse']
    )
    
    edge_model.summary() 
    
    edge_model.fit(full_edge_dataset.repeat(),
                   validation_data=full_edge_dataset,
                   steps_per_epoch=10,
                   epochs=50,
                   callbacks=[es])
    
    #Concatenate Both model's weight for each subgraph
    node_model_emb = node_model.layers[-1].get_weights()[0]
    edge_model_emb = edge_model.layers[-1].get_weights()[0] 
    
    #subgraph_emb = tf.concat([node_model_emb, edge_model_emb], axis=0).numpy()
    subgraph_emb = tf.matmul(node_model_emb, edge_model_emb, transpose_a=True).numpy()
  
    #subgraph_emb = tf.reduce_mean(subgraph_emb, axis=1)
    
    return subgraph_emb
    #np.save('graphtensors/citeseer_tensor', all_subgraphs_tensor)

In [ ]:
subgraph_corpus, subgraph_class = build_subgraph()

In [ ]:
def anchor_pairs():
    anchor_tensors = dict() 
    for i, sub in enumerate(subgraph_corpus):
        #lab = sub.pop(0)
        sub.pop(0)
        H = nx.complete_graph(sub)
        # if i < 20:
        #     nx.draw_spring(H)
        #     plt.show()
        lab = subgraph_class[i]
        anchor_tensors[lab] = graph_tfgnn(lab, H)
    
    return anchor_tensors

In [ ]:
def positive_pairs():
    G = create_graph()
    positive_tensors = dict()
    for i, sub in enumerate(subgraph_corpus):
        #labelz = sub.pop(0)
        sub.pop(0)
        H = nx.complete_graph(sub)
        
        edge_node = np.random.choice(list(H.nodes))
        out_nodes = set(G.nodes()) - set(sub)       
        random_node = np.random.choice(list(out_nodes))
        
        H.add_edge(edge_node, random_node)
        # if i < 20:
        #     nx.draw_spring(H)
        #     plt.show()
        lab = subgraph_class[i]
        positive_tensors[lab] = graph_tfgnn(lab, H)
    
    return positive_tensors